In [ ]:
import numpy as np
import pandas as pd

### CREATE DATA

In [ ]:
data_list = [['Sunny','Hot','High','Weak','No'],['Sunny','Hot','High','Strong','No'],['Overcast','Hot','High','Weak','Yes'],
            ['Rain','Mild','High','Weak','Yes'],['Rain','Cool','Normal','Weak','Yes'],['Rain','Cool','Normal','Strong','No'],
             ['Overcast','Cool','Normal','Strong','Yes'],['Sunny','Mild','High','Weak','No'],['Sunny','Cool','Normal','Weak','Yes'],
             ['Rain','Mild','Normal','Weak','Yes'],['Sunny','Mild','Normal','Strong','Yes'],['Overcast','Mild','High','Strong','Yes'],
             ['Overcast','Hot','Normal','Weak','Yes'],['Rain','Mild','High','Strong','No']]

columnNames = ['Outlook','Temp','Humidity','Wind','Play']
featureSpace_0 = ['sunny','overcast','rain']
featureSpace_1 = ['hot','mild','cool']
featureSpace_2 = ['normal','high']
featureSpace_3 = ['weak','strong']
featureSpace = [featureSpace_0,featureSpace_1,featureSpace_2,featureSpace_3]
data_arr = np.array(data_list)
func_Play = lambda val: 0 if val=='No' else 1
def func_Outlook(val) :
    if val.lower()=='sunny':
        return 0
    elif val.lower() =='overcast' :
        return 1
    else:
        return 2

def func_Temp(val) :
    if val.lower()=='hot':
        return 0
    elif val.lower() =='mild' :
        return 1
    else:
        return 2

def func_Wind(val) :
    if val.lower()=='weak':
        return 0
    else:
        return 1

def func_Humidity(val) :
    if val.lower()=='normal':
        return 0
    else:
        return 1

data_df = pd.DataFrame(data=data_arr,columns=columnNames) 
data_df['Play'] = data_df['Play'].apply(func_Play)
data_df['Outlook']= data_df['Outlook'].apply(func_Outlook)
data_df['Temp'] = data_df['Temp'].apply(func_Temp)
data_df['Wind'] = data_df['Wind'].apply(func_Wind)
data_df['Humidity'] = data_df['Humidity'].apply(func_Humidity)
data = data_df.to_numpy()

Nfeatures = 4
MaxFeatureSize = 3
treeChildren = -1*np.ones((Nfeatures,MaxFeatureSize),dtype=int)

In [ ]:
def getEntropy(data,Y,X=None,Xgiven=None):
    ''' 
    Calculates the conditional entropyies of the form H(Y), H(Y|X), H(Y|X1=x1,...,Xj=xj) and H(Y|Xi=xi,Xj=xj,X)
    INPUT: data : mxn numpy array of data
            Y: Index of the target vairable. 0<=Y<n
            X: The conditional vairable. 0<=X<n
            Xgiven = [(X1,x1),(X2,x2),....,(Xj,xj)]
    OUTPUT: H(Y|Xi=xi,...,Xj=xj,X)
    '''
    Ndata = data.shape[0]
    Yvalues = np.unique(data[:,Y])
    N_Yvalues = Yvalues.size
    
    if X==None and Xgiven==None: # Calculate the self-entropy H(Y)
        N=Ndata
        pYgivenX = np.zeros(N_Yvalues)
        for i in range(N_Yvalues):
            pYgivenX[i] = ((data[:,Y]==Yvalues[i]).sum())/N       
        HY = -pYgivenX.dot(np.log2(pYgivenX))
    elif Xgiven == None and X!=None: # Calculate simple conditional-entropy, H(Y|X)
        Xvalues = np.unique(data[:,X])
        N_Xvalues = Xvalues.size
        HY_givenX = np.zeros(N_Xvalues)
        pX = np.zeros(N_Xvalues)

        for i in range(N_Xvalues):            
            data_givenX = data[(data[:,X]==Xvalues[i])]
            pX[i] = ((data[:,X]==Xvalues[i]).sum())/Ndata
            Nx = data_givenX.shape[0]
            pYgivenX = np.zeros(N_Yvalues)
            for j in range(N_Yvalues):
                pYgivenX[j] = ((data_givenX[:,Y]==Yvalues[j]).sum())/Nx
            
            with np.errstate(divide='ignore'):
                logProb = np.log2(pYgivenX)
            logProb[np.isneginf(logProb)]=0
            HY_givenX[i] = -pYgivenX.dot(logProb)
        
        HY = pX.dot(HY_givenX)
    
    elif Xgiven!=None and X==None:# Calculate the conditional-entropy given some observations H(Y|X1=x1,...Xn=x)        
        HY_givenX = 0        
        data_filtered = data
        for x_given in Xgiven:
            data_temp = data_filtered[(data_filtered[:,x_given[0]]==x_given[1])]    
            data_filtered = data_temp        
        Ndata = data_filtered.shape[0]                                
        pYgivenX = np.zeros(N_Yvalues)
        
        for i in range(N_Yvalues):
            pYgivenX[i] = ((data_filtered[:,Y]==Yvalues[i]).sum())/Ndata
        
        with np.errstate(divide='ignore'):
            logProb = np.log2(pYgivenX)
        logProb[np.isneginf(logProb)]=0
        HY_givenX = -pYgivenX.dot(logProb)
        
        HY = HY_givenX
                
    
    else:# Calculate the conditional-entropy H(Y|Xi=xi,...,X)        
        Xvalues = np.unique(data[:,X])
        N_Xvalues = Xvalues.size
        HY_givenX = np.zeros(N_Xvalues)
        pX = np.zeros(N_Xvalues)

        data_filtered = data
        for x_given in Xgiven:
            data_temp = data_filtered[(data_filtered[:,x_given[0]]==x_given[1])]    
            data_filtered = data_temp
        
        Ndata = data_filtered.shape[0]        
        for i in range(N_Xvalues):                          
            data_givenX = data_filtered[(data_filtered[:,X]==Xvalues[i])]
            pX[i] = ((data_filtered[:,X]==Xvalues[i]).sum())/Ndata
            Nx = data_givenX.shape[0]            
            pYgivenX = np.zeros(N_Yvalues)
            for j in range(N_Yvalues):                
                if Nx == 0 :
                    pYgivenX[j]=0
                else:
                    pYgivenX[j] = ((data_givenX[:,Y]==Yvalues[j]).sum())/Nx
            
            with np.errstate(divide='ignore'):
                logProb = np.log2(pYgivenX)
            logProb[np.isneginf(logProb)]=0
            HY_givenX[i] = -pYgivenX.dot(logProb)
        
        HY = pX.dot(HY_givenX)
        
        
                
    return HY

In [ ]:
def findRoot(data,labelId,cols):
    '''
    Finds the root feature for the decision tree
    INPUT:  data - mxn numpy array of data , n = nFeatures + 1 (target)
            labelId - the id of the target/label
            cols - The names of all the n columns
    OUTPUT: root- the id of the root feature
    '''
    HY = getEntropy(data,Y=labelId)
    print('CONDITIONAL ENTROPIES: ')
    print(f'H(Y): {HY}')
    Nfeatures = len(cols)-1
    HY_Given_Feature = np.zeros(Nfeatures)
    for i in range(Nfeatures):
        HY_Given_Feature[i] = getEntropy(data,Y=labelId,X=i)
        print(f'H(Y|{columnNames[i]}) : {HY_Given_Feature[i]}')
    
    print('\nINFORMATION GAINS: ')

    for i in range(Nfeatures):    
        print(f'I(Y;{cols[i]}) : {HY- HY_Given_Feature[i]}')

    root = np.argmin(HY_Given_Feature)
    print(f'root : {root}')
    print(f'\nRoot feature: {cols[root]}')
    
    return root

In [ ]:
def findNextNode(data,currentRoot,featureSpace,labelId,treeChildren):
#     currentRoot = root
    Nfeatures = len(featureSpace)
    remainingFeatures = list(range(Nfeatures))
    remainingFeatures.remove(currentRoot)
    for i in range(len(featureSpace[currentRoot])):
        print(f'\nFINDING THE NEXT FEATURE TO CHECK AFTER {columnNames[currentRoot]}={featureSpace[currentRoot][i]}')
        currentRootValues = featureSpace[currentRoot]
        NcurrentFatures = len(currentRootValues)
        given=[(currentRoot,i)]
        HY = getEntropy(data,Y=labelId,X=None,Xgiven=given)
        M=100
        HY_Given_Feature = M*np.ones(len(remainingFeatures))
        print('CONDITIONAL ENTROPIES: ')
        print(f'H(Y|{columnNames[root]} = {currentRootValues[i]}): {HY}')
        if HY>0:
            for f in range(len(remainingFeatures)): 
                featureId = remainingFeatures[f]
                HY_Given_Feature[f] = getEntropy(data,Y=4,X=featureId,Xgiven=given)
                print(f'H(Y|{columnNames[currentRoot]} = {currentRootValues[i]},{columnNames[featureId]}) : {HY_Given_Feature[f]}')    
    
            print('INFORMATION GAINS: ')
            for f in range(len(remainingFeatures)): 
                featureId = remainingFeatures[f]
                print(f'I(Y;{columnNames[featureId]} = {currentRootValues[i]},{columnNames[featureId]}) : {HY - HY_Given_Feature[f]}')
        
            nextRoot = remainingFeatures[np.argmin(HY_Given_Feature)]
            print(f'Next Node after {columnNames[currentRoot]}= {featureSpace[currentRoot][i]}: {columnNames[nextRoot]} ')
            treeChildren[currentRoot,i] = nextRoot
    
        elif HY==0:
            m = np.argwhere(data[:,currentRoot]==i)[0,0]
            res = data[m,4]
            print(f'Leaf node! Decision: {res}')
            if res == 1:
                treeChildren[currentRoot,i] = 111
            else:
                treeChildren[currentRoot,i] = -111
    
    return treeChildren     


In [ ]:
root = findRoot(data,labelId=4,cols=columnNames)

In [ ]:
treeChildren = findNextNode(data=data,currentRoot=root,featureSpace=featureSpace,labelId=4,treeChildren=treeChildren)

In [ ]:
rowsDone = np.zeros(Nfeatures)
rowsDone[root] = 1
rootRow = treeChildren[root,:]
for featureValue in range(len(rootRow)):    
    data_filtered = data[(data[:,root]==featureValue)]
    cRoot = rootRow[featureValue]        
    if cRoot not in [-111,111,-1]:
        treeChildren = findNextNode(data=data_filtered,currentRoot=cRoot,featureSpace=featureSpace,labelId=4,treeChildren=treeChildren)
        rowsDone[cRoot] = 1
        print(rowsDone)
    
    

In [ ]:
treeChildren

In [ ]:
def predict(x,root,children):
    pred = None
    while pred!=111 and pred!= -111:
        res = int(children[root,x[root]])
        if res == 111 or res == -111:
            pred = res
        else:
            root = res
    return pred

In [ ]:
root = 0
y_pred = np.zeros(data.shape[0])
for m in range(data.shape[0]):
    instance = data[m,:]
    pred = predict(instance,root,treeChildren)
    if pred == 111:
        y_pred[m]=1
    else:
        y_pred[m]=0

    

In [ ]:
y_pred

In [ ]:
data[:,4]

In [ ]:
error_rate = (y_pred-data[:,4]).sum()/data.shape[0]

In [ ]:
error_rate

In [ ]:
import pandas as pd
csv_path = "C://Users/saurmisr/Downloads/titanic.csv"
data = pd.read_csv(csv_path)

In [ ]:
yTrain = data['Survived'].to_numpy()
PclassCol = data['Pclass'].apply(lambda X: 0 if X==1 else 1).to_numpy()
siblings_or_spousesCol = data['Siblings/Spouses Aboard'].apply(lambda X: 1 if X>0 else 0).to_numpy()
parents_or_childrenCol = data['Parents/Children Aboard'].apply(lambda X: 1 if X>0 else 0).to_numpy()
sexCol = data['Sex'].apply(lambda X: 1 if X=='male' else 0).to_numpy()
XTrainFeatureSet1 = np.column_stack((PclassCol,siblings_or_spousesCol,parents_or_childrenCol,sexCol))
(m,n) = XTrainFeatureSet1.shape
print(f'(m,n): {(m,n)}')

In [ ]:
data = np.column_stack((XTrainFeatureSet1,yTrain))
columnNames = ['PClass','Siblings/Spouse','Parents/Children','Sex','Survived']
featureSpace_0 = ['0','1']
featureSpace_1 = ['None','>0']
featureSpace_2 = ['None','>0']
featureSpace_3 = ['Male','Female']
featureSpace = [featureSpace_0,featureSpace_1,featureSpace_2,featureSpace_3]

Nfeatures = 4
MaxFeatureSize = 2
treeChildren = -1*np.ones((Nfeatures,MaxFeatureSize),dtype=int)

In [ ]:
root = findRoot(data,labelId=4,cols=columnNames)

In [ ]:
treeChildren = findNextNode(data=data,currentRoot=root,featureSpace=featureSpace,labelId=4,treeChildren=treeChildren)

In [ ]:
treeChildren

In [ ]:
rowsDone = np.zeros(Nfeatures)
rowsDone[root] = 1
rootRow = treeChildren[root,:]
for featureValue in range(len(rootRow)):    
    data_filtered = data[(data[:,root]==featureValue)]
    cRoot = rootRow[featureValue]        
    if cRoot not in [-111,111,-1]:
        treeChildren = findNextNode(data=data_filtered,currentRoot=cRoot,featureSpace=featureSpace,labelId=4,treeChildren=treeChildren)
        rowsDone[cRoot] = 1
        print(rowsDone)


In [ ]:
treeChildren

In [ ]:
root = 0
rowsDone[root] = 1
rootRow = treeChildren[root,:]
for featureValue in range(len(rootRow)):    
    data_filtered = data[(data[:,root]==featureValue)]
    cRoot = rootRow[featureValue]        
    if cRoot not in [-111,111,-1]:
        treeChildren = findNextNode(data=data_filtered,currentRoot=cRoot,featureSpace=featureSpace,labelId=4,treeChildren=treeChildren)
        rowsDone[cRoot] = 1
        print(rowsDone)


In [ ]:
treeChildren

In [ ]:
root = 0
y_pred = np.zeros(data.shape[0])
for m in range(data.shape[0]):
    print(m)
    instance = data[m,:]
    pred = predict(instance,root,treeChildren)
    if pred == 111:
        y_pred[m]=1
    else:
        y_pred[m]=0


In [ ]:
error_rate = np.abs(y_pred-data[:,4]).sum()/data.shape[0]
error_rate

In [85]:
treeChildren

array([[1, 2],
       [3, 3],
       [3, 3],
       [0, 0]])

In [ ]:
root = 3
m=0
instance = data[m,:]
pred = predict(instance,root,treeChildren)